In [114]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from scipy import signal
import pyedflib
from pyedflib import highlevel
import os
import h5py
import torch
import torchvision

In [115]:
#Constants 
dataPath = r"C:\Users\Humperdink2\Documents\github\NAThacks\Data\files"
h5Path = r"h5py/1personval2.h5"
numChannels = 64
labels = {
    "T0" : "rest",
    "T1" : ["leftHand", "bothHands"],
    "T2" : ["rightHand", "bothFeet"]
}
labelsDict = {
    "rest": 0,
    "leftHand": 1,
    "rightHand": 2,
    "bothHands": 3,
    "bothFeet": 4
}
frequency = 160
time_period = 640
prc_overlap = .90
chunk_size = 5000

In [116]:
samples = []
labels = []

In [ ]:
with h5py.File(h5Path, "w") as specs:
    specs.create_dataset("Samples",(0,81,31,64), maxshape=(None,81,31,64),compression="gzip")
    specs.create_dataset("Classes",(0,0), maxshape=(None,1), compression="gzip")
    print(f"h5 made, stores: Samples - {specs['Samples'].shape} | Classes - {specs['Classes'].shape}")

In [118]:
def storeDataSingle(dataSamples, dataLabels, size):
    with h5py.File(h5Path, "r+") as f:
        #Shuffling entries in the chunk
        tempLabels = []
        tempSamples = []
        order = np.random.permutation(np.arange(size))
        for pos in order:
            tempLabels.append(dataLabels[pos])
            tempSamples.append(dataSamples[pos])
        
        #Updating Samples
        df = f['Samples']
        current_shape = df.shape
        df.resize((current_shape[0] + size, 81, 31, 64))
        print(f" temp size is {np.array(tempSamples).shape}")
        df[current_shape[0]:, :] = tempSamples
        print(f"Samples adjusted to {df.shape}")

        # Update Lables
        labels = f['Classes']
        labels.resize((current_shape[0] + size,0))
        labels[current_shape[0]:,:] = tempLabels
        print(f"Classes adjusted to {labels.shape}")


In [ ]:
#Generating spectograms Images
person = 1
for folder in os.listdir(dataPath):
    for fileName in os.listdir(os.path.join(dataPath,folder)):

        #Skipping files .event files
        if fileName.find("event") != -1:
            continue

        #reading EDF file and extracting data
        filePath = os.path.join(dataPath,folder,fileName)
        annotations = ""
        file = ""

        #Getting the annotations and Data
        file = pyedflib.EdfReader(filePath)
        annotations = file.readAnnotations()
        file.close()

        signals, signal_headers, header = highlevel.read_edf(filePath)
        
        count = 0
        
        #Looping through the various 
        for i, period in enumerate(annotations[0]):
            #Getting signal data for 4 second period
            data = signals[:,int(annotations[0][i]):int(annotations[0][i]+time_period)]

            f, t, image = signal.spectrogram(data,frequency, nperseg = frequency, noverlap = frequency * prc_overlap)
            image = np.transpose(image,(1,2,0))

            #Assigning label to file
            label = annotations[2][i]

            #Categories of the different tests
            leftOrRight = ["03","04","07","08","11","12"]
            if label == "T1":
                if any(x == fileName[5:7] for x in leftOrRight):
                    labels.append(labelsDict["leftHand"])
                else: 
                    labels.append(labelsDict["bothHands"])
            elif label == "T2":
                if any(x == fileName[5:7] for x in leftOrRight):
                    labels.append(labelsDict["rightHand"])
                else:
                    labels.append(labelsDict["bothFeet"])
            else:
                labels.append(labelsDict["rest"])

            samples.append(image)
            if len(labels) == chunk_size:
                storeDataSingle(samples, labels, chunk_size)
                labels = []
                samples = []
                
            count += 1
    print(f"person {person} completed")
    person +=1

In [ ]:
storeDataSingle(samples, labels, len(samples))